# Content Based Filtering

In [2]:
import pandas as pd
from sklearn.neighbors import KDTree
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import random
random.seed(9001)
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
MoviesFeatures=pd.read_csv('../ml-latest-small/MoviesFeatures.csv')
#we use the data that we've genreted with the file Create_Movies_Features.ipynb


In [166]:
MoviesFeatures=MoviesFeatures.drop('Unnamed: 0',axis=1)

In [167]:
data=MoviesFeatures.dropna()
data.shape
data = data.reset_index(drop=True)
data.head(1)

,movieId,title,genre,year,writer,director,cast,rating,runtimes,countries,languages,production companies
0,1,Toy Story (1995),"Animation,Adventure,Comedy,Family,Fantasy",1995.0,"John Lasseter,Pete Docter,Andrew Stanton,Joe Ranft,Joss Whedon,Andrew Stanton,Joel Cohen,Alec So...","John Lasseter,","Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wallace Shawn,John Ratzenberger,Annie Potts,John Morr...",8.3,81,"USA,","English,","Pixar Animation Studios,Walt Disney Pictures,"


In [168]:
#we need to remove space from the name
for index, row in data.iterrows():
    data.ix[index,'writer']=data.ix[index,'writer'].replace(" ", "")
    data.ix[index,'director']=data.ix[index,'director'].replace(" ", "")
    data.ix[index,'cast']=data.ix[index,'cast'].replace(" ", "")
    data.ix[index,'production companies']=data.ix[index,'production companies'].replace(" ", "")

## recommendation based on genre

In [271]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['genre'].tolist())
Xarray=X.toarray()
cosine_similarities1 = cosine_similarity(Xarray, Xarray)
def cb_similaire_item1(id,n):#similarity based on genre
    id=id-1
    similar_indices = cosine_similarities1[id].argsort()[:-(n+1):-1]
    similar_items = [(cosine_similarities1[id][i], data['movieId'][i]) for i in similar_indices]
    return similar_items[1:]
#cb_similaire_item1(1,20)

In [207]:
#data.query('movieId==1 | movieId==2090 |  movieId==2355 | movieId==1031 | movieId==1030' )

## recommendation based on writer

In [212]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['writer'].tolist())
Xarray=X.toarray()
cosine_similarities2 = cosine_similarity(Xarray, Xarray)
def cb_similaire_item2(id,n):#similarity based on writer
    id=id-1
    similar_indices = cosine_similarities2[id].argsort()[:-(n+1):-1]
    similar_items = [(cosine_similarities2[id][i], data['movieId'][i]) for i in similar_indices]
    return similar_items[1:]
#cb_similaire_item2(1,20)

In [205]:
#data.query('movieId==1 | movieId==2355 |  movieId==1604' )[["movieId","title","writer"]]

## recommendation based on director

In [213]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['director'].tolist())
Xarray=X.toarray()
cosine_similarities3 = cosine_similarity(Xarray, Xarray)
def cb_similaire_item3(id,n):#similarity based on director
    id=id-1
    similar_indices = cosine_similarities3[id].argsort()[:-(n+1):-1]
    similar_items = [(cosine_similarities3[id][i], data['movieId'][i]) for i in similar_indices]
    return similar_items[1:]
#cb_similaire_item3(1,20)

In [203]:
#(data.query('movieId==1 | movieId==2355 ' ))[['movieId','title','director']]

## recommendation based on Countries

In [214]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['countries'].tolist())
Xarray=X.toarray()
cosine_similarities4 = cosine_similarity(Xarray, Xarray)
def cb_similaire_item4(id,n):#similarity based on countries
    id=id-1
    similar_indices = cosine_similarities4[id].argsort()[:-(n+1):-1]
    similar_items = [(cosine_similarities4[id][i], data['movieId'][i]) for i in similar_indices]
    return similar_items[1:]
#cb_similaire_item4(1,20)

## Recommendation based on Production Companies

In [219]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['production companies'].tolist())
Xarray=X.toarray()
cosine_similarities5 = cosine_similarity(Xarray, Xarray)
def cb_similaire_item5(id,n):#similarity based on production companies
    id=id-1
    similar_indices = cosine_similarities5[id].argsort()[:-(n+1):-1]
    similar_items = [(cosine_similarities5[id][i], data['movieId'][i]) for i in similar_indices]
    return similar_items[1:]
#cb_similaire_item5(1,20)

In [201]:
#(data.query('movieId==1 | movieId==2355 | movieId==1020 ' ))[['movieId','title','production companies']]

## recommendation based on all

In [268]:
l,c=data.shape

def content_based(i):#i est l'indice du film pour lequel on veut chercher les item similaire
    cb1=cb_similaire_item1(i,l)#similarity based on genre
    cb2=cb_similaire_item2(i,l)#similarity based on writer
    cb3=cb_similaire_item3(i,l)#similarity based on director
    cb4=cb_similaire_item4(i,l)#similarity based on countries
    cb5=cb_similaire_item5(i,l)#similarity based on production companies
    
    df1=pd.DataFrame(cb1,columns=['score_genre','movieId'])
    df2=pd.DataFrame(cb2,columns=['score_writer','movieId'])
    df3=pd.DataFrame(cb3,columns=['score_director','movieId'])
    df4=pd.DataFrame(cb4,columns=['score_countries','movieId'])
    df5=pd.DataFrame(cb5,columns=['score_production_companies','movieId'])
    
    df_all=df1.join(df2.set_index('movieId'), on='movieId').join(df3.set_index('movieId'), on='movieId').join(df4.set_index('movieId'), on='movieId').join(df5.set_index('movieId'), on='movieId')
    df_all["score_all"]=df_all["score_genre"]+df_all["score_writer"]+df_all["score_director"]+df_all["score_countries"]+df_all["score_production_companies"]
    df_all=df_all.sort_values(by='score_all',ascending=False)
    
    return df_all

In [272]:
content_based(1).head(4)

,score_genre,movieId,score_writer,score_director,score_countries,score_production_companies,score_all
1,1.000000,2355,0.67082,0.707107,1.0,1.000000,4.377927
24,0.800000,1282,0.00000,0.000000,1.0,0.707107,2.507107
73,0.670820,1020,0.00000,0.000000,1.0,0.707107,2.377927
11,0.845154,1566,0.00000,0.000000,1.0,0.500000,2.345154
